In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
/kaggle/input/testing-data-set/testing_data_set.csv


In [2]:
# lib import 
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# load dataset 
data = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
data

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [3]:
# idk apparently you have to choose a column as the one that is the closting price so i chose near_price i could be wrong
target_column_name = 'target'

dd = data.fillna(0)

# yeah define the column i chose
target = dd[target_column_name]
dd = dd.drop(target_column_name, axis=1)
dd

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.000000,0.000000,0.999812,60651.50,1.000026,8493.03,1.000000,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,0.000000,0.000000,0.999896,3233.04,1.000660,20605.09,1.000000,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,0.000000,0.000000,0.999403,37956.00,1.000298,18995.00,1.000000,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,0.000000,0.000000,0.999999,2324.90,1.000214,479032.40,1.000000,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,0.000000,0.000000,0.999394,16485.54,1.000016,434.10,1.000000,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,26454,480_540_198


In [4]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(dd, target, test_size=0.1, random_state=42)

# input features standardized 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# reshape input data for model assuming a time series format
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [5]:
# define lstm model 
model = Sequential()
model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(16, activation='relu', return_sequences=True))
model.add(Dense(1))
model.compile(optimizer='Adagrad', loss='mean_absolute_error')

In [6]:
# train
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
73660/73660 [==============================] - 912s 12ms/step - loss: 6.4076
Epoch 2/10
73660/73660 [==============================] - 880s 12ms/step - loss: 6.4075
Epoch 3/10
73660/73660 [==============================] - 912s 12ms/step - loss: 6.4075
Epoch 4/10
73660/73660 [==============================] - 904s 12ms/step - loss: 6.4075
Epoch 5/10
73660/73660 [==============================] - 895s 12ms/step - loss: 6.4075
Epoch 6/10
73660/73660 [==============================] - 897s 12ms/step - loss: 6.4075
Epoch 7/10
73660/73660 [==============================] - 946s 13ms/step - loss: 6.4075
Epoch 8/10
73660/73660 [==============================] - 931s 13ms/step - loss: 6.4074
Epoch 9/10
73660/73660 [==============================] - 971s 13ms/step - loss: 6.4075
Epoch 10/10
73660/73660 [==============================] - 962s 13ms/step - loss: 6.4074


In [7]:
# evaluate
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# predict
predictions = model.predict(X_test)

16369/16369 [==============================] - 70s 4ms/step - loss: 6.4067
Test Loss: 6.40668249130249
16369/16369 [==============================] - 74s 4ms/step
